# Introduction

In [1]:
import sklearn
import pandas as pd
import numpy as np
import csv
import scipy.stats as stats
from statistics import pstdev
from statistics import mean
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from itertools import product
import copy


In [2]:
df = pd.read_excel('sri_spfeas_v3.xlsx', sheet_name=0)

In [3]:
#Ensure that the GN_Code field is of type int and make it the index
df['gid'] = df['gid'].astype(int)
df = df.set_index('gid')

In [4]:
df.head()

,fid,gnd_n,gnd_c,fourier_sc31_mean_mean,fourier_sc31_mean_std,fourier_sc31_mean_sum,fourier_sc31_variance_mean,fourier_sc31_variance_std,fourier_sc31_variance_sum,fourier_sc51_mean_mean,...,sfs_sc71_min_ll_sum,sfs_sc71_roa_mean,sfs_sc71_roa_std,sfs_sc71_roa_sum,sfs_sc71_std_mean,sfs_sc71_std_std,sfs_sc71_std_sum,sfs_sc71_w_mean_mean,sfs_sc71_w_mean_std,sfs_sc71_w_mean_sum
gid,,,,,,,,,,,,,,,,,,,,,
43,0,Karagaskada South,2148065,69.453586,3.136141,2.376702e+05,18.629130,0.976240,6.374888e+04,78.493900,...,3933,136.394391,0.938602,466878,16.233955,3.648450,55568.828125,1.228705,0.707158,4205.858398
564,1,Panvila,2115070,59.535572,7.845283,1.400872e+06,17.086123,2.204723,4.020365e+05,69.651317,...,40076188,137.910516,2.093903,3245724,14.498129,5.118067,341213.468750,2.838843,3.471181,66812.164062
7917,2,Ganepalla,9227135,50.730112,5.682764,3.625884e+06,14.450501,2.109653,1.032835e+06,60.710447,...,7900571136,139.556597,2.014015,9974110,9.856942,7.202832,704475.625000,3.388618,5.864246,242184.515625
2263,3,Akkaraipattu 11,5236065,73.670361,1.821871,9.731855e+04,18.843218,1.231690,2.489189e+04,82.544710,...,1331,136.077853,0.267941,180031,10.558998,4.329045,13969.554688,0.514668,0.239074,680.906372
8,4,Delgasthenna,2109125,66.878083,2.710715,2.657735e+05,18.708531,1.015577,7.434770e+04,75.957167,...,4809,136.681028,1.377590,542487,15.218429,4.465160,60401.945312,1.180995,0.868924,4687.369141


In [5]:
df.shape

(14021, 426)

In [6]:
depvar = pd.read_csv("sri_ghs.csv")

In [7]:
depvar.shape

(14021, 13)

In [8]:
depvar['gid'] = depvar['gid'].astype(int)
depvar = depvar.set_index('gid')

In [17]:
depvar.head()

,gnd_n,gnd_c,prov_c,dis_c,dsd_c,GN_Area,lka_count,lka_sum,lka_mean,ghs_count,ghs_sum,ghs_mean
gid,,,,,,,,,,,,
43,Karagaskada South,2148065,2,21,2148,337190.326,34.0,1158.294682,34.067491,235,635,2.702128
564,Panvila,2115070,2,21,2115,2316039.056,232.0,2185.832124,9.421690,1626,6326,3.890529
7917,Ganepalla,9227135,9,92,9227,7042858.261,706.0,1836.615909,2.601439,4929,6929,1.405762
2263,Akkaraipattu 11,5236065,5,52,5236,130560.240,11.0,1248.624039,113.511276,89,5889,66.168539
8,Delgasthenna,2109125,2,21,2109,390874.578,38.0,1701.824509,44.784855,274,2874,10.489051


In [18]:
merged = df.merge(depvar, left_on='gnd_c', right_on='gnd_c', how='outer')

In [19]:
stats.describe(merged['ghs_mean'])

DescribeResult(nobs=14021, minmax=(1.0, 101.0), mean=12.392446343557369, variance=572.2103732068704, skewness=2.44133226610882, kurtosis=4.875565418551456)

In [20]:
#Get the list of dependent variables from the DataFrame to store in list y_vars
y_var = list(merged.axes[1])[436]
#Initialize a dictionary to store independent variables for each dependent variable 
y_dict = {}
#Get a list of all independent variables from the DataFrame in list all_x
all_x = list(merged.axes[1])[3:426]
y_var

'ghs_mean'

In [21]:
all_x

['fourier_sc31_mean_mean',
 'fourier_sc31_mean_std',
 'fourier_sc31_mean_sum',
 'fourier_sc31_variance_mean',
 'fourier_sc31_variance_std',
 'fourier_sc31_variance_sum',
 'fourier_sc51_mean_mean',
 'fourier_sc51_mean_std',
 'fourier_sc51_mean_sum',
 'fourier_sc51_variance_mean',
 'fourier_sc51_variance_std',
 'fourier_sc51_variance_sum',
 'fourier_sc71_mean_mean',
 'fourier_sc71_mean_std',
 'fourier_sc71_mean_sum',
 'fourier_sc71_variance_mean',
 'fourier_sc71_variance_std',
 'fourier_sc71_variance_sum',
 'gabor_sc3_filter_1_mean',
 'gabor_sc3_filter_1_std',
 'gabor_sc3_filter_1_sum',
 'gabor_sc3_filter_10_mean',
 'gabor_sc3_filter_10_std',
 'gabor_sc3_filter_10_sum',
 'gabor_sc3_filter_11_mean',
 'gabor_sc3_filter_11_std',
 'gabor_sc3_filter_11_sum',
 'gabor_sc3_filter_12_mean',
 'gabor_sc3_filter_12_std',
 'gabor_sc3_filter_12_sum',
 'gabor_sc3_filter_13_mean',
 'gabor_sc3_filter_13_std',
 'gabor_sc3_filter_13_sum',
 'gabor_sc3_filter_14_mean',
 'gabor_sc3_filter_14_std',
 'gabor_sc3

# With the whole dataset

Check Relationship with pearson Correlation

In [22]:

#Variable x is an empty list that will cotnain all significant (p < 0.05) independent variables
x = []
for x_var in all_x:
    #Calculate the Pearson statistics, returns the Pearson value and p value
    p = stats.pearsonr(merged[x_var],merged[y_var])
    # print back for mike
    print (y_var + " , " + x_var + " , " +  str(p[0]) + " , " + str(p[1]))
    #If p < 0.05 append to list x
    if p[1] < 0.05:
        x.append([x_var,abs(p[0])])
        
#List x is made into a DataFrame which is sorted by the absolute values of the Pearson values
x_df = pd.DataFrame(x,columns=["x_var","abs_r2"]).sort_values("abs_r2",ascending=False)

#The dependent variable dictionary is given an entry where the key is the name of the dependent variable
# and the value is a list of top 200 most significant values

y_dict[y_var] = list(x_df["x_var"][0:294])
#Print out each dependent variable and the number of x values that remain to check completion
for key in y_dict.keys():
    print(key,len(y_dict[key]))


ghs_mean , fourier_sc31_mean_mean , 0.7091570337201256 , 0.0
ghs_mean , fourier_sc31_mean_std , -0.2729866171423062 , 4.327445325382961e-238
ghs_mean , fourier_sc31_mean_sum , -0.13569405284815386 , 1.3172600036163263e-58
ghs_mean , fourier_sc31_variance_mean , 0.5352570739373398 , 0.0
ghs_mean , fourier_sc31_variance_std , -0.2381526041439328 , 5.075104732805422e-180
ghs_mean , fourier_sc31_variance_sum , -0.13692813314696234 , 1.180862546425526e-59
ghs_mean , fourier_sc51_mean_mean , 0.6933276662890375 , 0.0
ghs_mean , fourier_sc51_mean_std , -0.3008458243860631 , 3.5480408285857185e-291
ghs_mean , fourier_sc51_mean_sum , -0.13529429510440508 , 2.8634289095873156e-58
ghs_mean , fourier_sc51_variance_mean , 0.5443127257599042 , 0.0
ghs_mean , fourier_sc51_variance_std , -0.2691636504807443 , 2.832855756166806e-231
ghs_mean , fourier_sc51_variance_sum , -0.13651887932328216 , 2.634218953669792e-59
ghs_mean , fourier_sc71_mean_mean , 0.6806648731844532 , 0.0
ghs_mean , fourier_sc71_mean

ghs_mean , sfs_sc31_max_ll_sum , -0.1309758300740416 , 1.0826550883762378e-54
ghs_mean , sfs_sc31_mean_mean , 0.7343035524673873 , 0.0
ghs_mean , sfs_sc31_mean_std , 0.1332095860800519 , 1.5806910966013785e-56
ghs_mean , sfs_sc31_mean_sum , -0.12483822574538066 , 8.209817689181215e-50
ghs_mean , sfs_sc31_min_ll_mean , -0.041895235562067513 , 6.9568634610948e-07
ghs_mean , sfs_sc31_min_ll_std , -0.0628976226696078 , 9.02477277201974e-14
ghs_mean , sfs_sc31_min_ll_sum , -0.10940611718900443 , 1.3464105932270564e-38
ghs_mean , sfs_sc31_roa_mean , -0.6367636161936754 , 0.0
ghs_mean , sfs_sc31_roa_std , 0.1527249932732354 , 6.2623676493055824e-74
ghs_mean , sfs_sc31_roa_sum , -0.1329203489191375 , 2.7436750352174295e-56
ghs_mean , sfs_sc31_std_mean , 0.35198267934913136 , 0.0
ghs_mean , sfs_sc31_std_std , -0.6383413879442018 , 0.0
ghs_mean , sfs_sc31_std_sum , -0.15231438375572617 , 1.5426676943611624e-73
ghs_mean , sfs_sc31_w_mean_mean , -0.17121795973016257 , 1.0414275099089096e-92
ghs_me

### Correlation absolute value

In [23]:
x_df

,x_var,abs_r2
186,hog_sc7_kurtosis_mean,0.793919
281,mean_sc3_variance_mean,0.768957
174,hog_sc5_kurtosis_mean,0.766846
192,hog_sc7_skew_mean,0.748357
371,sfs_sc31_mean_mean,0.734304
63,gabor_sc3_variance_mean,0.726578
33,gabor_sc3_filter_14_mean,0.726192
36,gabor_sc3_filter_2_mean,0.725072
180,hog_sc5_skew_mean,0.724473
27,gabor_sc3_filter_12_mean,0.724289


In [24]:
y = y_var
#Initialize the output dictionary, Y_D, with each key being a dependent variable and the values being the results of the analyses
Y_D = {}
#Dictionary Models is used to store each result object for later use if needed
Models ={}
#for each dependent variable y in the list of all dependent values, calibrate the model

#Add new key to the output dictionary where y is the dependent variable curently being processed
#and the values are empty for now
Y_D[y]={}
#Get independent variables from the variable dictionary and store in list x_vars
x_vars = y_dict[y]
#Create a new dataframe for scaled and centered values
scale_df = pd.DataFrame()
#Scale and center the values
scale_df[y] = scale(merged[y_var], with_mean=True, with_std=True)
for x in x_vars:
    scale_df[x] = scale(merged[x], with_mean=True, with_std=True)
result = ElasticNetCV(max_iter=1e8,
                    alphas = [0.0005, 0.001, 0.01, 0.03, 0.05, 0.1],
                    l1_ratio =[.1, .5, .7, .9, .95, .99, 1],
                    verbose= False,
                    n_jobs = -1, 
                    cv=5, 
                    selection = 'random',
                    fit_intercept=False)
#Fit the model with the scaled data
result.fit(scale_df[x_vars],scale_df[y])
#Append the model to the Models dictionary
Models[y] = result


/Users/adbe/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/Users/adbe/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [25]:
result

ElasticNetCV(alphas=[0.0005, 0.001, 0.01, 0.03, 0.05, 0.1], copy_X=True, cv=5,
       eps=0.001, fit_intercept=False,
       l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], max_iter=100000000.0,
       n_alphas=100, n_jobs=-1, normalize=False, positive=False,
       precompute='auto', random_state=None, selection='random',
       tol=0.0001, verbose=False)

In [26]:
opt_alpha, opt_l1_ratio = result.alpha_, result.l1_ratio_
#Print update to ensure that the script is progressing properly
print("R2: {:.2f} Alpha: {} l1_ratio: {}"
      .format(result.score(scale_df[x_vars],scale_df[y]),
              result.alpha_, result.l1_ratio_))
#Record the overall R squared score and optimal alpha and l1 ratio values and store them in the output dictionary
Y_D[y]['Total R2'] = result.score(scale_df[x_vars],scale_df[y])
Y_D[y]['Alpha'] = opt_alpha
Y_D[y]['l1_ratio'] = opt_l1_ratio
#Create a list R2s to store out of sample R squared values
R2s = []
#Specify the number of trials to run
trials = 10
#Run the number of trials specified in trials, for each trial 34% of the observations are randomly selected to train the model
#Testing is done on the remaining 66% of observations and the R squared values are recorded
for i in range(trials):    
    X_train, X_test, y_train, y_test = train_test_split(scale_df[x_vars], scale_df[y], test_size=0.66)
    test_regr = ElasticNetCV(max_iter=1e8,
                    alphas = [opt_alpha],
                    l1_ratio =[opt_l1_ratio],
                    n_jobs = -1, 
                    cv=5, 
                    selection = 'random',
                    fit_intercept=False)
    test_regr.fit(X_train,y_train)
    R2s.append(test_regr.score(X_test,y_test))

#print("Mean R2: {:.2f} StDev: {:.4f}".format(mean(R2s),pstdev(R2s)))
#Record the out of sample R squared values
Y_D[y]['Sampling']={'trials':trials,'R2':mean(R2s),'StDev':pstdev(R2s),'R2s':R2s}
coefs = [i for i in zip(list(scale_df[x_vars].axes[1]),result.coef_)]
remaining = [i for i in coefs if abs(i[1])>0.0]
Y_D[y]["Coefficients"]=remaining

R2: 0.82 Alpha: 0.01 l1_ratio: 0.1


In [27]:
y_vars = y_var
Y_D[y_var]

{'Total R2': 0.8216149480519188,
 'Alpha': 0.01,
 'l1_ratio': 0.1,
 'Sampling': {'trials': 10,
  'R2': 0.8146108544396765,
  'StDev': 0.0026515664964481753,
  'R2s': [0.8160652506968857,
   0.8194653905136783,
   0.814700660544187,
   0.8167673517499754,
   0.8104926208459506,
   0.8147658220525544,
   0.8169645147712374,
   0.8121438346860141,
   0.8115563890136596,
   0.8131867095226221]},
 'Coefficients': [('hog_sc7_kurtosis_mean', 0.20065283078019489),
  ('mean_sc3_variance_mean', 0.10317386650156868),
  ('hog_sc5_kurtosis_mean', 0.06969838218349107),
  ('hog_sc7_skew_mean', 0.011735608446083201),
  ('sfs_sc31_mean_mean', 0.00921849225645454),
  ('gabor_sc3_filter_14_mean', 0.01987217012268506),
  ('hog_sc5_skew_mean', 0.02102412306322438),
  ('gabor_sc3_filter_12_mean', 0.02651967731808506),
  ('ndvi_sc3_variance_mean', 0.50888864518865),
  ('hog_sc3_kurtosis_mean', 0.04202893537918417),
  ('hog_sc3_variance_std', -0.06273521639699982),
  ('fourier_sc51_mean_mean', 0.0549052251060

In [33]:
df2 = merged[merged['ghs_mean']>=10]

In [34]:
df2.head()

,fid,gnd_n_x,gnd_c,fourier_sc31_mean_mean,fourier_sc31_mean_std,fourier_sc31_mean_sum,fourier_sc31_variance_mean,fourier_sc31_variance_std,fourier_sc31_variance_sum,fourier_sc51_mean_mean,...,prov_c,dis_c,dsd_c,GN_Area,lka_count,lka_sum,lka_mean,ghs_count,ghs_sum,ghs_mean
3,3,Akkaraipattu 11,5236065,73.670361,1.821871,9.731855e+04,18.843218,1.231690,24891.890625,82.544710,...,5,52,5236,130560.240,11.0,1248.624039,113.511276,89,5889,66.168539
4,4,Delgasthenna,2109125,66.878083,2.710715,2.657735e+05,18.708531,1.015577,74347.703125,75.957167,...,2,21,2109,390874.578,38.0,1701.824509,44.784855,274,2874,10.489051
6,6,Santhiveli,5110065,67.196778,9.780546,1.303147e+06,19.325224,2.683984,374774.062500,77.062510,...,5,51,5110,1907449.335,138.0,2688.887347,19.484691,1343,33943,25.274013
7,7,Nawayalathenna,2112235,66.658046,3.350067,4.336772e+05,19.087925,1.260433,124186.039062,75.722050,...,2,21,2112,640533.554,62.0,2096.300272,33.811295,449,16449,36.634744
14,14,Kattankudy division 1 South,5124010,75.893946,3.789219,1.028363e+05,18.512343,1.067105,25084.224609,84.043323,...,5,51,5124,133748.420,13.0,1779.547729,136.888287,94,8494,90.361702


In [36]:
#df4.shape, pop_density.shape
df2.shape

(3131, 437)

In [25]:
# import matplotlib.pyplot as plt
# plt.subplots(figsize=(15,15))
# y = pop_density
# x = df4['PCNT_built']
# plt.scatter(x, y)

# z = np.polyfit(x, y, 1)
# p = np.poly1d(z)
# plt.plot(x,p(x),"r--")
# plt.show()

In [37]:
df2.columns

Index(['fid', 'gnd_n_x', 'gnd_c', 'fourier_sc31_mean_mean',
       'fourier_sc31_mean_std', 'fourier_sc31_mean_sum',
       'fourier_sc31_variance_mean', 'fourier_sc31_variance_std',
       'fourier_sc31_variance_sum', 'fourier_sc51_mean_mean',
       ...
       'prov_c', 'dis_c', 'dsd_c', 'GN_Area', 'lka_count', 'lka_sum',
       'lka_mean', 'ghs_count', 'ghs_sum', 'ghs_mean'],
      dtype='object', length=437)